# TUIA - Computer Vision - Trabajo Practico Final

Alumno: Cesar Julian Donnarumma.

Año: 2024.

## Ejercicio 04: Evaluacion

Se debe ejecutar el notebook de evaluación provisto con el fin de evaluar el modelo sobre el conjunto de prueba y obtener las mediciones para verificar los requisitos de funcionamiento que permiten la evaluación al momento del examen.

Se espera que al modificar la ruta de almacenamiento del dataset, se ejecuten todas las funciones que nos permitirán evaluar el trabajo.

### Importaciones:

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import os
import shutil
import json
from ultralytics import YOLO
from google.colab import drive
import matplotlib.pyplot as plt
import cv2

Montamos el Drive para trabajar con el modelo entrenado y con los datos de prueba

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


### Ajustes Iniciales

In [5]:
# Nombre del alumno
student_name = "cesar_donnarumma"

# Ruta al archivo de pesos
model_path = "/content/drive/MyDrive/TUIA_Computer_Vision/TP_02/modelos/03 - yolov8x_aumentacion_x2/weights/best.pt"

# Ruta al directorio que contiene las imagenes
imgs_dir = "/content/drive/MyDrive/TUIA_Computer_Vision/TP_02/evaluacion/data/eval/images/val"

# Ruta al directorio de destino de las detecciones
base_dir = "/content/drive/MyDrive/TUIA_Computer_Vision/TP_02/evaluacion/data/out"
dets_dir = os.path.join(base_dir, student_name)

In [6]:
# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)

os.makedirs(dets_dir)

### Cálculo del envido

Carga del modelo que vamos a utilizar

In [7]:
model = YOLO(model_path)

Imagenes de evaluacion

In [8]:
imagenes_evaluacion = [ img for img in os.listdir(imgs_dir) ]
imagenes_evaluacion = sorted(imagenes_evaluacion)

Funcion de json generico

In [9]:
# Funcion que genera estructura de diccionario general para todas las cartas
def generar_carta_generica():

  carta_generica = {}
  carta_generica['total_cards'] = 0
  carta_generica['cards'] = {}
  carta_generica['cards']['E'] = []
  carta_generica['cards']['C'] = []
  carta_generica['cards']['B'] = []
  carta_generica['cards']['O'] = []
  carta_generica['points'] = 0
  carta_generica['figure'] = 'N/A'

  return carta_generica

Predicciones, logica del envido, creacion de .txt y de .json.

Se incluye a modo control bloques de comentario que dicen ----- Control ----- | ----- Fin de control ------ que se pueden comentar si molestan.

In [11]:
# Diccionario que sera el json con los resultados
envido_json = {}
# Recorremos cada imagen
for imagen in imagenes_evaluacion:
  # Hacemos inferencia una a una (el resultado queda en results[0])
  results = model(os.path.join(imgs_dir, imagen), verbose=True, conf=0.5)
  # Guardamos los resultados de la deteccion en el .txt que pide el enunciado
  results[0].save_txt(txt_file=os.path.join(dets_dir, os.path.splitext(imagen)[0] + '.txt'), save_conf=True)
  # Agregamos al diccionario, futuro json, la seccion correspondiente a la carta en cuestion
  envido_json[imagen] = generar_carta_generica()
  # Creamos una lista con indices correspondientes a cartas detectadas
  cartas = list(results[0].boxes.cls.cpu().numpy())
  # Guardamos en una variable la cantidad de cartas
  total_cartas = len(cartas)
  # Guardamos en el json la cantidad de cartas detectadas en la imagen en la seccion correspondiente
  envido_json[imagen]['total_cards'] = total_cartas
  # Si se detectan cartas
  if total_cartas > 0:
    # Buscamos el nombre de las cartas segun su indice en el diccionario segun su indice (filtrando el comodin)
    cartas = [results[0].names[carta] for carta in cartas if results[0].names[carta] != 'J']
    # ------------------- Control --------------------------
    # Lista de detecciones:
    print(f'\nCartas detectadas: {cartas}\n')
    # ----------------- Fin Control ------------------------
    # Completamos la lista de palos del json con los numero correspondiente a las detecciones
    for carta in cartas:
      envido_json[imagen]['cards'][carta[-1]].append(int(carta[:-1]))
    # Si hay 3 cartas en la imagen y no hay ni 8 ni 9 podemos considerar que es una mano de truco
    if (total_cartas == 3) and not (any(carta.startswith(("8", "9")) for carta in cartas)):
      # Calculamos la cantidad de cartas por palos
      cantidad_espada = len(envido_json[imagen]['cards']['E'])
      cantidad_copa = len(envido_json[imagen]['cards']['C'])
      cantidad_basto = len(envido_json[imagen]['cards']['B'])
      cantidad_oro = len(envido_json[imagen]['cards']['O'])
      # Guardamos en una lista
      cantidades = [ cantidad_espada, cantidad_copa, cantidad_basto, cantidad_oro ]
      # Ordenamos la lista para quedarnos luego con el mayor
      cantidades = sorted(cantidades)
      # Creamos un diccionario que nos ayudara a encontrar cual es el palo que mas cartas tiene
      cantidad_cartas = {}
      # Metemos las cantidades como clave y el nombre del palo como valor
      cantidad_cartas[cantidad_espada] = 'E'
      cantidad_cartas[cantidad_copa] = 'C'
      cantidad_cartas[cantidad_basto] = 'B'
      cantidad_cartas[cantidad_oro] = 'O'
      # De la lista con las cantidades por palo nos quedamos con la mas grande
      maximo = cantidades[-1]
      # Con el numero maximo como clave buscamos en el diccionario que nos indicara a que palo corresponde. Con dicho
      # palo buscamos en el json todos los numeros del palo detectados. Nos quedamos con los menores a o iguales a 7
      nuevas = [ carta for carta in envido_json[imagen]['cards'][cantidad_cartas[maximo]] if carta <= 7 ]
      # Ordenamos los numeros
      nuevas = sorted(nuevas)
      # Si hay 3 cartas del mismo palo
      if maximo == 3:
        # Si hay 3 o 2 menores a 8
        if (len(nuevas) == 3) or (len(nuevas) == 2):
          puntos = nuevas[-1] + nuevas[-2] + 20 # Los puntos son 20 + la mayor + la siguiente
          envido_json[imagen]['figure'] = cantidad_cartas[maximo] # El palo es el palo que tiene la mayor cantidad de cartas
        elif len(nuevas == 1): # Si hay 1 carta sola menor a 8
          puntos = nuevas[-1] + 20 # Los puntos son 20 + la unica carta
          envido_json[imagen]['figure'] = cantidad_cartas[maximo] # El palo es el palo que tiene la mayor cantidad de cartas
        else:
          puntos = 20 # Si no hay ninguna carta menor a 8 son 20 puntos fijos
      # Si hay 2 cartas del mismo palo
      elif maximo == 2:
        # Si hay 2 cartas menores a 8
        if len(nuevas) == 2:
          puntos = nuevas[-1] + nuevas[-2] + 20 # Los puntos son 20 + las dos cartas
          envido_json[imagen]['figure'] = cantidad_cartas[maximo]
        elif len(nuevas == 1): # Si hay una sola carta menor a 8
          puntos = nuevas[-1] + 20 # Los puntons son 20 mas la unica carta
          envido_json[imagen]['figure'] = cantidad_cartas[maximo]
        else:
          puntos = 20 # 2 cartas del mismo palo mayores a 9 son 20 puntos fijos
      # Si hay todas cartas de distinto palo
      else:
        # Nos quedamos con las menores a 8
        nuevas = [ carta for carta in cartas if int(carta[:-1]) <= 7 ]
        # Ordenamos ascendentemente
        nuevas = sorted(nuevas)
        # Si no hay ninguna menor a 8
        if len(nuevas) == 0:
          puntos = 0 # Los puntos son 0 fijos
        else: # Sino
          puntos = int(nuevas[-1][:-1]) # Los puntos son los de la mas grande
          envido_json[imagen]['figure'] = nuevas[-1][-1] # El palo es el ultimo caracter de dicha carta
      # Asignamos los puntos en el json donde corresponde
      envido_json[imagen]['points'] = puntos
    # Si hay menos o mas de 3 cartas u hay 8 o 9 entre ellas:
    else:
      print(f'No se puede cantar envido, no es una mano de truco.\n')
    # ------------------- Control: --------------------------
    # Grafico en pantalla con detecciones:
    annotated_image = results[0].plot()
    img_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,8))
    plt.imshow(img_rgb)
    plt.title(f'Imagen evaluacion: {os.path.split(imagen)[1]}')
    plt.axis('off')
    plt.show()
    # Diccionario/json:
    print(f'\n{envido_json[imagen]}\n')
    # ----------------- Fin Control: ------------------------
    print('-------------'*5 + '\n')

Output hidden; open in https://colab.research.google.com to view.

### Escritura del archivo envido.json

In [12]:
# Se abre el archivo en la ruta indicada (si no existe se crea) como escritura
with open(os.path.join(dets_dir, "envido.json"), "w") as jf:
    # Se escribe la informacion de envido_json en el archivo abierto con 4 espacio de indentacion
    json.dump(envido_json, jf, indent=4)